In [16]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-zwnj-base")

loading configuration file https://huggingface.co/HooshvareLab/bert-fa-zwnj-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ba5343a1268e870ceacba2041556e22a4480d6af08e330b0650f96c96a5e50b4.967063c40b5c987b6e0d2a6d1944963280b9a53e5821fcb5379494a07dc9408d
Model config BertConfig {
  "_name_or_path": "HooshvareLab/bert-fa-zwnj-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}

loading file https://huggingface.co/Ho

In [19]:
import numpy as np
import pandas as pd

f = open("Train.txt", "r", encoding = 'utf-8')
lines = f.readlines()

train_tagged = []

for line in lines:
    if line == '\n':
        continue
    train_tagged.append(tuple(line.strip().split()))

x = []
y = []
sen_x = []
sen_y = []
for tup in train_tagged:
    x.append(tup[0])
    y.append(tup[1])
    if tup[0] == '.':
        sen_x.append(x)
        sen_y.append(y)
        x = []
        y = []

train_df = pd.DataFrame()
train_df['tags'] = sen_y
train_df['tokens'] = sen_x

tags = {tag for word,tag in train_tagged}

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(list(tags))
train_df['tags']=train_df['tags'].apply(lambda r:le.transform(r))

In [20]:
from datasets import Dataset
dataset_train = Dataset.from_pandas(train_df.iloc[:7000])
dataset_val = Dataset.from_pandas(train_df.iloc[7000:])

In [21]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                #label_ids.append(-100)
                pass
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                #label_ids.append(-100)
                pass
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [22]:
tokenized_dataset_train = dataset_train.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

In [23]:
tokenized_dataset_val = dataset_val.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [24]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [25]:
num_classes = 23

In [26]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("HooshvareLab/bert-fa-zwnj-base", num_labels=num_classes)

loading configuration file https://huggingface.co/HooshvareLab/bert-fa-zwnj-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ba5343a1268e870ceacba2041556e22a4480d6af08e330b0650f96c96a5e50b4.967063c40b5c987b6e0d2a6d1944963280b9a53e5821fcb5379494a07dc9408d
Model config BertConfig {
  "_name_or_path": "HooshvareLab/bert-fa-zwnj-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LA

In [27]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [28]:
tokenized_dataset_train

Dataset({
    features: ['tags', 'tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 7000
})

In [29]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [30]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tags, tokens. If tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7000
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4380


Epoch,Training Loss,Validation Loss
1,No log,1.095488
2,1.396700,1.063477
3,0.985600,0.780659
4,0.803000,0.663933
5,0.677800,0.603565
6,0.558000,0.555061
7,0.491300,0.569683
8,0.400400,0.516012
9,0.400400,0.512336
10,0.323200,0.515467


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tags, tokens. If tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1722
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tags, tokens. If tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1722
  Batc

TrainOutput(global_step=4380, training_loss=0.6668469799163679, metrics={'train_runtime': 1800.5295, 'train_samples_per_second': 38.877, 'train_steps_per_second': 2.433, 'total_flos': 4464122622163200.0, 'train_loss': 0.6668469799163679, 'epoch': 10.0})

In [31]:
f = open("Test.txt", "r", encoding = 'utf-8')
lines = f.readlines()

test_tagged = []

for line in lines:
    if line == '\n':
        continue
    test_tagged.append(tuple(line.strip().split()))

x = []
y = []
sen_x = []
sen_y = []
for tup in test_tagged:
    x.append(tup[0])
    y.append(tup[1])
    if tup[0] == '.':
        sen_x.append(x)
        sen_y.append(y)
        x = []
        y = []

test_df = pd.DataFrame()
test_df['tags'] = sen_y
test_df['tokens'] = sen_x

test_df['tags']=test_df['tags'].apply(lambda r:le.transform(r))

In [32]:
dataset_test = Dataset.from_pandas(test_df)

In [33]:
tokenized_dataset_test = dataset_test.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [34]:
predictions = trainer.predict(tokenized_dataset_test)

The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tags, tokens. If tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 9279
  Batch size = 16


In [35]:
preds = []
for i,sentence in enumerate(tokenized_dataset_test['tags']):
  l = len(sentence)
  p = predictions.predictions[i][:l]
  t = np.argmax(p, axis=-1)
  preds.append(t)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(list(np.concatenate(preds).flat), list(np.concatenate(tokenized_dataset_test['tags']).flat))

0.8186766862875687